**1**. (100 points)

Write a predicate function `is_prime` that efficiently checks whether a number is prime. Use this to write a second function `primes_between` that returns the prime numbers between two integers as a `numpy` array.

- Do this in regular Python
- Accelerate using `numba` (serial version)
- Accelerate using `numba` (parallel version)
- Accelerate using `cython` (serial version)
- Accelerate using `cython` (parallel version)
- Run the serial version of the `numba` `primes_between` function in parallel using
    - `multiprocessing`
    - `joblib`
    - `ipyparallel`
- Report the speed-up of the `numba` and `cython` versions using `timeit` in a table for the numbers between 0 and 1,000,000

In [2]:
import numpy as np
import numba

In [53]:
%load_ext cython

In [5]:
m = 0
n = 1_000_000

## Regular Python version

In [50]:
def is_prime(n):
    """Check if n is prime."""
    
    if n == 2:
        return True
    elif n % 2 == 0 or n < 2:
        return False
    else:
        for i in range(3, 1 + int(np.sqrt(n)), 2):
            if n % i == 0:
                return False
    return True

def primes_between(m, n):
    """Return primes between m and n."""
    
    return np.array([i for i in range(m, n) if is_prime(i)])

In [52]:
primes_between(0, 100)

array([ 2,  3,  5,  7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59,
       61, 67, 71, 73, 79, 83, 89, 97])

In [53]:
t1 = %timeit -r3 -n3 primes_between(m, n)

2.88 s ± 25.7 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


## Numba (serial)

In [55]:
@numba.jit(nopython=True)
def is_prime_numba_serial(n):
    """Check if n is prime."""
    
    if n == 2:
        return True
    elif n % 2 == 0 or n < 2:
        return False
    else:
        for i in range(3, 1 + int(np.sqrt(n)), 2):
            if n % i == 0:
                return False
    return True

@numba.jit(nopython=True)
def primes_between_numba_serial(m, n):
    """Return primes between m and n."""
    
    return np.array([i for i in range(m, n) if is_prime_numba_serial(i)])

In [56]:
t2 = %timeit -r3 -n3 primes_between_numba_serial(m, n)

154 ms ± 65.4 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


## Numba (parallel)

In [49]:
@numba.jit(nopython=True)
def is_prime_numba_parallel(n):
    """Check if n is prime."""
    
    if n == 2:
        return True
    elif n % 2 == 0 or n < 2:
        return False
    else:
        for i in range(3, 1 + int(np.sqrt(n)), 2):
            if n % i == 0:
                return False
    return True

@numba.jit(nopython=True, parallel=True)
def primes_between_numba_parallel(m, n):
    """Return primes between m and n."""
    
    idx = np.zeros(n-m)
    for k in numba.prange(m, n):
        if is_prime_numba_parallel(k):
            idx[k-m] = 1
    return np.arange(m, n)[np.nonzero(idx)]

In [50]:
primes_between_numba_parallel(0, 10)

array([2, 3, 5, 7])

In [51]:
t3 = %timeit -r3 -n3 primes_between_numba_parallel(m, n)

26.4 ms ± 3.18 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


## Cython (serial)

In [83]:
%%cython -a 

import cython
from libc.math cimport sqrt

@cython.cdivision
cdef bint is_prime_cython_serial(int n):
    """Check if n is prime."""
    
    cdef int i
    cdef int m = <int>sqrt(n)
    
    if n == 2:
        return True
    elif n % 2 == 0 or n < 2:
        return False
    else:
        for i in range(3, 1 + m, 2):
            if n % i == 0:
                return False
    return True

def primes_between_cython_serial(int m, int n):
    """Return primes between m and n."""
    
    cdef int i
    cdef list primes = []
    
    for i in range(m, n):
        if is_prime_cython_serial(i):
            primes.append(i)
    return primes

In [76]:
primes_between_cython_serial(0, 10)

[2, 3, 5, 7]

In [84]:
t3 = %timeit -r3 -n3 primes_between_cython_serial(m, n)

80.3 ms ± 4.98 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


## Cython parallel

In [111]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp --force -I /usr/local/opt/libomp/include -L /usr/local/opt/libomp/lib

import cython
import numpy as np
from numpy cimport ndarray as ar
from libc.math cimport sqrt
from cython.parallel import parallel, prange

@cython.cdivision
cdef bint is_prime_cython_parallel(int n) nogil:
    """Check if n is prime."""
    
    cdef int i
    if n == 2:
        return True
    elif n % 2 == 0 or n < 2:
        return False
    else:
        for i in range(3, 1 +  <int>sqrt(n), 2):
            if n % i == 0:
                return False
    return True

@cython.boundscheck(False)
def primes_between_cython_parallel(int m, int n):
    """Return primes between m and n."""
    
    cdef int i, k
    cdef ar[int] idx = np.zeros(n-m, dtype='int')
    
    with cython.nogil, parallel():
        for i in prange(m, n):
            if is_prime_cython_parallel(i):
                k = i - m
                idx[k] = 1
    return np.arange(m, n)[np.nonzero(idx)]

In [112]:
primes_between_cython_parallel(0, 10)

ValueError: Buffer dtype mismatch, expected 'int' but got 'long'